In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
skl.set_config(print_changed_only=False)

General idea
import in the dataset

inspect data, partition out train and test

make a pipeline
get the list of hyperparameters using get_params
use gridsearch to optimise these params

fit the model

In [4]:
drop_cols = ["Name", "Ticket","Cabin","Parch"] # Remove these data that doesn't seem to be very necessary
test = pd.read_csv('titanic_dataset/test.csv').drop(drop_cols, axis=1)
train = pd.read_csv('titanic_dataset/train.csv').drop(drop_cols,axis=1)
train_X = train.drop("Survived", axis=1)
train_y = train.loc[:, "Survived"] # this is a pandas series

col_names = train_X.columns.to_list()

In [5]:
pd.DataFrame(train_y)

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [6]:
# Column transform
transform_cols = ["Sex", "Embarked"]
trf = ColumnTransformer(
    transformers=[
        ('encode', OneHotEncoder(), transform_cols),
        ("no_modifications", "passthrough", [x for x in col_names if x not in transform_cols])
    ]
)
trf.fit(train_X)
train_X = trf.transform(train_X)
test = trf.transform(test)


In [7]:
print(train_X[0])

[ 0.    1.    0.    0.    1.    0.    1.    3.   22.    1.    7.25]


In [8]:
# Create model
clf = XGBClassifier(learning_rate=0.001, n_estimators=600, objective='binary:logistic', nthread=4)

number_of_folds = 7
param_comb = 50

skf = StratifiedKFold(n_splits=number_of_folds, shuffle=True, random_state=1001)
# Call skf.split to return training indices for the stratified kfolds

# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=params,
    n_iter=param_comb,
    scoring='roc_auc',
    n_jobs=5,
    verbose=1,
    cv=skf.split(train_X, train_y),
    random_state=1001)

random_search.fit(train_X, train_y) # took 40 seconds

# try grid search - how mcuh slower is it - took 5 mins on mbp
# grid = GridSearchCV(estimator=clf, param_grid=params, scoring='roc_auc', n_jobs=8,cv=skf.split(train_X, train_y))
# grid.fit(train_X, train_y)


Fitting 7 folds for each of 50 candidates, totalling 350 fits


/Users/john/Desktop/machine-learning/.venv/lib/python3.11/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/john/Desktop/machine-learning/.venv/lib/python3.11/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/john/Desktop/machine-learning/.venv/lib/python3.11/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/john/Desktop/machine-learning/.venv/lib/python3.11/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/john/Desk

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x14f4f5240>,
                   error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           im...
                                           multi_strategy=None,
                                           n_estimators=600, n_jobs=None,
                                           nthread=4, num_parallel_tree=None, ...),
                   n_iter=50, n_jobs=5,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   pre_dispatch='2*n_jobs', random_state=1001, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=1)

In [9]:
searchcv = random_search
best_estimator = searchcv.best_estimator_
print(f"Score = {searchcv.best_score_}")
y_pred = best_estimator.predict(test)

Score = 0.8716132178980927
